In [1]:
import numpy as np
from matplotlib import pyplot
from scipy.integrate import solve_ivp
from scipy.optimize import brentq

import math

from IPython.display import HTML
from ipywidgets.widgets import interact, interactive, IntSlider, FloatSlider, Layout

%matplotlib inline

style = {'description_width': '150px'}
slider_layout = Layout(width='99%')

In [2]:
def simulate(initialPopulation, infectedOnDayZero, earlyExponentialRate, meanDurationOfInfectionInDays, timespanInDays):
    uninfectedOnDayZero = initialPopulation - infectedOnDayZero
    growthRate = earlyExponentialRate / initialPopulation
    decayRate = 1 / meanDurationOfInfectionInDays
    
    def odeRhs(timeInDays, dependentValues):
        uninfected, infected = dependentValues
        infectionRate = growthRate * uninfected * infected
        rateOfChangeOfUninfected = - infectionRate
        rateOfChangeOfInfected = infectionRate - decayRate * infected
        return (rateOfChangeOfUninfected, rateOfChangeOfInfected)
    
    odeSolution = solve_ivp(odeRhs, (0, timespanInDays), ((uninfectedOnDayZero, infectedOnDayZero)), "LSODA")

    
    days = odeSolution.t
    populations = odeSolution.y
    
    uninfectedAtPeakInfection = uninfectedOnDayZero / (earlyExponentialRate * meanDurationOfInfectionInDays)
    
    asymptoticUninfected = brentq(
        lambda asymptoticUninfectedEstimate:
            uninfectedOnDayZero - asymptoticUninfectedEstimate - uninfectedAtPeakInfection * (math.log(uninfectedOnDayZero) - math.log(asymptoticUninfectedEstimate)),
        1, uninfectedAtPeakInfection)

        
    # Shamelessly copied from: https://elc.github.io/posts/ordinary-differential-equations-with-python/


    
    fig, subplots = pyplot.subplots(1, 2, figsize=(15, 10))
    
    subplotAxes = subplots[0]
    
    subplotAxes.plot(days, populations[0], label='Uninfected(t)')
    subplotAxes.plot(days, populations[1], label='Infected(t)')
    

    step = 15
    rotation = "horizontal"
    
    subplotAxes.set_xticklabels(np.arange(0, timespanInDays + 1, step, dtype=np.int), rotation=rotation)
    subplotAxes.set_xticks(np.arange(0, timespanInDays + 1, step))
        
    subplotAxes.set_xlim([0, timespanInDays])
    subplotAxes.set_xlabel('Time')
    subplotAxes.set_ylabel('Population')
    subplotAxes.set_yscale('log')
    subplotAxes.axhline(asymptoticUninfected, label = "Final uninfected: {}".format(asymptoticUninfected), linestyle = "dashed", color = "green")
    subplotAxes.axhline(uninfectedAtPeakInfection, label = "Uninfected at peak infection: {}".format(uninfectedAtPeakInfection), linestyle = "dashed", color = "red")
    subplotAxes.legend(loc='best')
    subplotAxes.grid()
    
    pyplot.tight_layout()
    pyplot.show()

    
interact(simulate,
         initialPopulation = FloatSlider(min = 1, max = 6E+7, value = 6E+7, description = 'Initial population.', style=style, layout=slider_layout, continuous_update=False),
         infectedOnDayZero = FloatSlider(min = 1, max = 1000, value = 10, description = 'Initial number of infected people.', style=style, layout=slider_layout, continuous_update=False),
         earlyExponentialRate = FloatSlider(min = 0, max = 3, value = 0.248562
, description = 'Early exponential rate.', style=style, layout=slider_layout, continuous_update=False),
         meanDurationOfInfectionInDays = FloatSlider(min = 1, max = 100, value = 28, description = 'Mean duration of infection.', style=style, layout=slider_layout, continuous_update=False),
         timespanInDays = IntSlider(min = 10, max = 365, value = 300, description = 'Timespan in days.', style=style, layout=slider_layout, continuous_update=False));
        

interactive(children=(FloatSlider(value=60000000.0, continuous_update=False, description='Initial population.'…